In [6]:
import pandas as pd
import datetime
import os
import json
pd.options.mode.chained_assignment = None  # default='warn'

In [126]:
from multiprocessing.sharedctypes import Value


class GeneradorPlanillaFinanzas:
    def __init__(self):
        pass

    def correr_programa(self):
        self.diccionario_dfs = self.identificar_tipo_de_archivo_y_cargar_dfs()
        self.dfs_limpias = self.limpiar_dfs(self.diccionario_dfs)
        df_completa, df_columnas_utiles = self.unir_dfs(self.dfs_limpias)

        self.guardar_dfs(df_completa, df_columnas_utiles)
    
    def identificar_tipo_de_archivo_y_cargar_dfs(self):
        identificadores_archivo = {'SII': 'SII', 
                                 'Acepta': 'ACEPTA',
                                 'TURBO': 'TURBO',
                                 'SCI': 'SCI',
                                 'Sigfe': 'SIGFE'}

        llaves_identificadoras = list(identificadores_archivo.keys())
                
        diccionario_dfs = {'SII': None,
                           'ACEPTA': None, 
                           'TURBO': None, 
                           'SCI': None,
                           'SIGFE': None}

        for nombre_archivo in os.listdir('input_cortados'):
            nombre_archivo = os.path.join('input_cortados', nombre_archivo)
            if ('.xlsx' in nombre_archivo) or ('.xls' in nombre_archivo) or ('.csv' in nombre_archivo):
                for llave in llaves_identificadoras:
                    if llave in nombre_archivo:
                        identificador_archivo = identificadores_archivo[llave]
                        print(f'Leyendo {nombre_archivo}, es del tipo: {identificador_archivo}')
                        diccionario_dfs[identificador_archivo] = pd.read_excel(nombre_archivo)
                        break

        return diccionario_dfs
    
    # Solo se limpian las que no son del SII.    
    def limpiar_dfs(self, diccionario_dfs):
        for nombre_tabla, df in diccionario_dfs.items():
            if nombre_tabla == 'ACEPTA':
                df.rename(columns = {'emisor': 'RUT Emisor', 'folio': 'Folio'}, inplace = True)
            
            elif nombre_tabla == 'TURBO':
                df.rename(columns = {'Rut': 'RUT Emisor', 'Folio': 'Folio_interno', 'NºDoc.': 'Folio'}, inplace = True)

            elif nombre_tabla == 'SCI':
                df.rename(columns = {'Rut Proveedor': 'RUT Emisor', 'Numero Documento': 'Folio'}, inplace = True)
            
            elif nombre_tabla == 'SIGFE':
                df['RUT Emisor'] = df['Principal'].str.split(' ')[0]
                df.rename(columns = {'Folio': 'Folio_interno', 'Número ': 'Folio'}, inplace = True)

                mask_debe = (df['Debe'] != 0)
                mask_haber = (df['Haber'] != 0)

                df.loc[mask_debe, 'Folio_interno PAGO'] = df['Folio_interno'][mask_debe]
                df.loc[mask_debe, 'Fecha PAGO'] = df['Fecha'][mask_debe]

                df.loc[mask_haber, 'Folio_interno DEVENGO'] = df['Folio_interno'][mask_haber]                
                df.loc[mask_haber, 'Fecha DEVENGO'] = df['Fecha'][mask_haber]

            
            df['RUT Emisor'] = df['RUT Emisor'].str.replace('.', '').upper().strip()
            
            df.columns = df.columns + f' {nombre_tabla}'

            df['llave_id'] = df[f'RUT Emisor {nombre_tabla}'].astype(str) + df[f'Folio {nombre_tabla}'].astype(str)
            df.set_index('llave_id', drop = True, inplace = True)
        
        # for columna in df.columns:
        #     verificador = columna.lower()
        #     if 'fecha' in verificador:
        #         df[columna] = df[columna].date

        return diccionario_dfs
    
    def buscador_de_nota_de_credito_asociada(self, llave_id, columna_referencias):
        try:
            indice = columna_referencias.index(llave_id)
        except ValueError:
            indice = None
        
        return indice
    
    def unir_dfs(self, diccionario_dfs_limpias):
        lista_dfs_secuenciales = list(diccionario_dfs_limpias.values())
        df_izquierda = lista_dfs_secuenciales[0]

        for df_derecha in lista_dfs_secuenciales[1:]:
            df_izquierda = pd.merge(df_izquierda, df_derecha, how = 'left', left_index = True, right_index = True)
    
        df_izquierda = df_izquierda[~df_izquierda.index.duplicated(keep = 'first')]

        df_izquierda = self.calcular_tiempo_8_dias(df_izquierda)
        df_izquierda = self.obtener_referencias_nc(df_izquierda)
        df_izquierda = self.obtener_columnas_necesarias(df_izquierda)

        return df_izquierda

    
    def calcular_tiempo_8_dias(self, df_izquierda):
        mask_no_devengadas = pd.isna(df_izquierda['Fecha DEVENGO SIGFE'])

        df_izquierda['Fecha Docto SII'] = pd.to_datetime(df_izquierda['Fecha Docto SII'], dayfirst = True)
        df_izquierda.loc[mask_no_devengadas, 'tiempo_diferencia SII'] = pd.to_datetime('today') - df_izquierda[mask_no_devengadas]['Fecha Docto SII']
        df_izquierda.loc[mask_no_devengadas, 'esta_al_dia'] = df_izquierda[mask_no_devengadas]['tiempo_diferencia SII'] <= datetime.timedelta(8)

        return df_izquierda
    
    def obtener_referencias_nc(self, df_izquierda):
        mask_notas_de_credito = df_izquierda['Tipo Doc SII'] == 61
        df_izquierda.loc[mask_notas_de_credito, 'Factura que referencia'] = df_izquierda[mask_notas_de_credito]['RUT Emisor SII'] + df_izquierda[mask_notas_de_credito]['referencias ACEPTA'].apply(lambda x: json.loads(x)[0]['Folio'] if type(x) == str else 'NO ESTA EN ACEPTA')

        for referencia in df_izquierda['Factura que referencia']:
            if type(referencia) == str:
                if not('NO ESTA EN ACEPTA' in referencia):
                    nc = df_izquierda[df_izquierda['Factura que referencia'] == referencia].index[0]
                    df_izquierda.loc[referencia, 'NC Asociada'] = nc
        
        df_izquierda['Factura que referencia'] = df_izquierda['Factura que referencia'].apply(lambda x: x.split('-')[1][1:] if type(x) == str else None)
        df_izquierda['NC Asociada'] = df_izquierda['NC Asociada'].apply(lambda x: x.split('-')[1][1:] if type(x) == str else None)

        return df_izquierda
    
    def obtener_columnas_necesarias(self, df_izquierda):
        columnas_a_ocupar = ['Tipo Doc SII', 'RUT Emisor SII', 'Razon Social SII', 'Folio SII', 'Fecha Docto SII',
                           'Monto Exento SII', 'Monto Neto SII', 'Monto IVA Recuperable SII', 'Monto Total SII',
                           'publicacion ACEPTA', 'estado_acepta ACEPTA', 'estado_sii ACEPTA', 'estado_nar ACEPTA', 'estado_devengo ACEPTA', 'folio_oc ACEPTA', 'folio_rc ACEPTA', 'fecha_ingreso_rc ACEPTA', 'folio_sigfe ACEPTA', 'tarea_actual ACEPTA',
                           'estado_cesion ACEPTA', 'Fecha DEVENGO SIGFE', 'Folio_interno DEVENGO SIGFE', 'Fecha PAGO SIGFE', 'Folio_interno PAGO SIGFE', 'Fecha Recepción SCI', 'Registrador SCI', 'Articulo SCI', 'N° Acta SCI', 'Ubic. TURBO', 'NºPresu TURBO', 'Folio_interno TURBO', 'NºPago TURBO',
                           'tiempo_diferencia SII', 'esta_al_dia', 'Factura que referencia', 'NC Asociada']

        df_util = df_izquierda[columnas_a_ocupar]

        return df_izquierda, df_util
    
    def guardar_dfs(self, df_completa, df_columnas_utiles):
        fecha_actual = str(pd.to_datetime('today')).split(' ')[0]
        nombre_archivo = f'PLANILLA DE CONTROL AL {fecha_actual}.xlsx'

        if nombre_archivo in os.listdir():
            with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'a', if_sheet_exists = 'overlay') as writer:
                df_columnas_utiles.to_excel(writer)
        else:
            df_columnas_utiles.loc[:, 'Observaciones'] = None
            with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'w') as writer:
                df_columnas_utiles.to_excel(writer)

In [127]:
programa = GeneradorPlanillaFinanzas()
programa.correr_programa()

Leyendo input_cortados\Acepta 2019-2022_cortado.xlsx, es del tipo: ACEPTA
Leyendo input_cortados\SCI 2021-2022_cortado.xlsx, es del tipo: SCI
Leyendo input_cortados\Sigfe_2019-2022_cortado.xlsx, es del tipo: SIGFE
Leyendo input_cortados\SII 2019-2021_cortado.xlsx, es del tipo: SII FACTURAS ELECTRONICAS/RECLAMADAS/NC
Leyendo input_cortados\TURBO 2019-2022_cortado.xlsx, es del tipo: TURBO


AttributeError: 'str' object has no attribute 'date'